In [1]:
import os

# Set the number of threads for all relevant libraries
num_threads = "8"
os.environ["OMP_NUM_THREADS"] = num_threads
os.environ["OPENBLAS_NUM_THREADS"] = num_threads
os.environ["MKL_NUM_THREADS"] = num_threads
os.environ["VECLIB_MAXIMUM_THREADS"] = num_threads
os.environ["NUMEXPR_NUM_THREADS"] = num_threads

import torch
import numpy as np
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device}")

if device.type == 'cpu':
    torch.set_num_threads(8)
    print(f"Limiting cpu threads to: {torch.get_num_threads()}")

Using cpu
Limiting cpu threads to: 8


In [2]:
import numpy as np
import pyvista as pv
import torch
import torch.nn as nn

# =============================================================================
# 1. POINT GENERATION FROM STL FILES
# =============================================================================

# --- Parameters ---
# FIX: Reduced the number of points to prevent memory overload.
# Start with smaller numbers and increase them later if needed.
n_collocation_points = 5000
n_boundary_points = 1000

# --- Load and Process Geometries ---
try:
    print("Loading mesh files...")
    coil_mesh = pv.read('coil.stl')
    housing_mesh = pv.read('housing.stl') # Assuming this is the correct filename

    # FIX: A more robust method to create a single, watertight surface
    print("Combining and cleaning meshes...")
    # 1. Merge the meshes into a single volumetric representation.
    combined_volume = pv.merge([coil_mesh, housing_mesh])
    # 2. Extract the outer surface of the combined volume.
    surface = combined_volume.extract_surface()
    # 3. Clean the surface (merges duplicate points, etc.) and then fill holes.
    combined_mesh = surface.clean().fill_holes(hole_size=200.0)
    print("Mesh processing complete.")

except FileNotFoundError as e:
    print(f"Error: {e}. Make sure your STL files are in the correct directory.")
    exit()

## --- Generate Collocation Points ---
#print("Generating collocation points...")
#combined_bounds = combined_mesh.bounds
#random_points_np = np.random.uniform(
#    low=[combined_bounds[0], combined_bounds[2], combined_bounds[4]],
#    high=[combined_bounds[1], combined_bounds[3], combined_bounds[5]],
#    size=(n_collocation_points * 2, 3)  # Oversample
#)
#
## Correct usage of select_enclosed_points:
## 1. Create a PolyData object from the random points.
#point_cloud = pv.PolyData(random_points_np)
## 2. Call select_enclosed_points ON the point_cloud, passing the mesh as the surface.
#selection = point_cloud.select_enclosed_points(combined_mesh, tolerance=1e-6)
## 3. FIX: Get the boolean mask from the result's point_data.
#selected_mask = selection.point_data['SelectedPoints'].astype(bool)
## 4. Use this mask to select points from the original numpy array.
#collocation_points_np = random_points_np[selected_mask]


print("Generating collocation points in batches...")
combined_bounds = combined_mesh.bounds
collocation_points_list = []
batch_size = 2000 # Process 2000 random points at a time

# Keep generating batches until we have enough points
while len(collocation_points_list) < n_collocation_points:
    # Generate one batch of random points
    random_points_np = np.random.uniform(
        low=[combined_bounds[0], combined_bounds[2], combined_bounds[4]],
        high=[combined_bounds[1], combined_bounds[3], combined_bounds[5]],
        size=(batch_size, 3)
    )

    # This is the corrected way to call select_enclosed_points:
    # 1. Create a PolyData object from the random points.
    point_cloud = pv.PolyData(random_points_np)
    # 2. Call the function ON the point_cloud, passing the mesh as the surface.
    selection = point_cloud.select_enclosed_points(combined_mesh, tolerance=1e-6)
    # 3. Get the boolean mask from the result.
    selected_mask = selection.point_data['SelectedPoints'].astype(bool)
    
    # Add the points that were inside the mesh to our list
    enclosed_points = random_points_np[selected_mask]
    if len(enclosed_points) > 0:
        collocation_points_list.extend(enclosed_points)
    
    # Optional: Print progress
    print(f"\r  Found {len(collocation_points_list)} / {n_collocation_points} points...", end="")

# Combine all found points from the batches into a single array
collocation_points_np = np.array(collocation_points_list)
# Trim to the exact number desired, in case we overshot
if len(collocation_points_np) > n_collocation_points:
    collocation_points_np = collocation_points_np[:n_collocation_points]

#==========================================================================================================

# --- Generate Boundary Points ---
#print("Generating fixed boundary points...")
#all_surface_vertices = combined_mesh.points
#num_to_sample = min(n_boundary_points * 5, len(all_surface_vertices))
#random_indices = np.random.choice(len(all_surface_vertices), num_to_sample, replace=False)
#surface_points_array = all_surface_vertices[random_indices]
#
#bottom_z = combined_bounds[4]
#fixed_boundary_points_np = surface_points_array[
#    np.isclose(surface_points_array[:, 2], bottom_z)
#]

print("\nGenerating fixed boundary points in batches...")
bottom_z = combined_bounds[4]
fixed_boundary_points_list = []
total_surface_vertices = combined_mesh.points

# Keep sampling until we find enough points on the bottom surface
while len(fixed_boundary_points_list) < n_boundary_points:
    # Take a random sample of the mesh's existing vertices
    random_indices = np.random.choice(len(total_surface_vertices), batch_size, replace=False)
    surface_points_batch = total_surface_vertices[random_indices]
    
    # Filter this smaller batch for points on the bottom
    bottom_points_batch = surface_points_batch[
        np.isclose(surface_points_batch[:, 2], bottom_z)
    ]

    # Add any found points to our list
    if len(bottom_points_batch) > 0:
        fixed_boundary_points_list.extend(bottom_points_batch)

    print(f"\r  Found {len(fixed_boundary_points_list)} / {n_boundary_points} points...", end="")


# Combine all found points and trim to the exact number
fixed_boundary_points_np = np.array(fixed_boundary_points_list)
if len(fixed_boundary_points_np) > n_boundary_points:
    fixed_boundary_points_np = fixed_boundary_points_np[:n_boundary_points]

#==========================================================================================

print(f"\nGenerated {len(collocation_points_np)} collocation points.")
print(f"Generated {len(fixed_boundary_points_np)} fixed boundary points.")

# --- Convert to Tensors for PINN ---
collocation_points = torch.tensor(collocation_points_np, dtype=torch.float32, requires_grad=True)
fixed_boundary_points = torch.tensor(fixed_boundary_points_np, dtype=torch.float32, requires_grad=True)

#==================making a simpler vtk file for use in visualising the result of the PINN==================

print("\nCreating and saving simplified mesh for visualization...")
vis_mesh = combined_mesh.decimate(target_reduction=0.0)
vis_mesh_filename = 'visualization_mesh.vtk'
vis_mesh.save(vis_mesh_filename)
print(f"Simplified mesh saved to '{vis_mesh_filename}'")

del combined_mesh

# --- Visualization (to verify the result) ---
#print("\nDisplaying generated points...")
#plotter = pv.Plotter()
#plotter.add_mesh(combined_mesh, style='wireframe', color='gray', opacity=0.1)
#plotter.add_points(collocation_points_np, color='blue', point_size=2, render_points_as_spheres=True, label='Collocation Points')
#plotter.add_points(fixed_boundary_points_np, color='red', point_size=4, render_points_as_spheres=True, label='Fixed Boundary Points')
#plotter.add_legend()
#plotter.show()

print("\nPoint generation complete and visualization skipped.")


Loading mesh files...
Combining and cleaning meshes...
Mesh processing complete.
Generating collocation points in batches...
  Found 5063 / 5000 points...
Generating fixed boundary points in batches...
  Found 1009 / 1000 points...
Generated 5000 collocation points.
Generated 1000 fixed boundary points.

Creating and saving simplified mesh for visualization...
Simplified mesh saved to 'visualization_mesh.vtk'

Point generation complete and visualization skipped.


In [3]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# =============================================================================
# 1. DEFINE MATERIAL AND FORCE PROPERTIES FOR THE NEW PROBLEM
# =============================================================================

# Define a Single Material (e.g., Aluminum)
E_MATERIAL = 69e9  # Pascals
NU_MATERIAL = 0.33

# Define Forces
# NOTE: The coordinates are based on your CadQuery script (in mm).
# The top surface of the top plate is at z = coil_height + plate_thickness = 85 + 5 = 90.
x_forces = torch.tensor([
    [0.0, 100.0, 85.0/2.0]#, # Force on top plate
    #[0.0, -100.0, 85.0/2.0] # Force on top plate
])
P_forces = torch.tensor([
    [0.0, 5000.0, 0.0]#, # 500N force +y direction
    #[0.0, -5000.0, 0.0]  # 500N force -y direction
])




# =============================================================================
# 2. PINN IMPLEMENTATION
# =============================================================================

# Your Neural Network class
# NOTE: The name is changed to reflect the new problem, but the structure is identical.
class PINN_Helmholtz(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 3)
        )

    def forward(self, x):
        # NOTE: The hard-coded boundary condition `(x**2) * nn_output` from the
        # cantilever beam is removed, as it does not apply to this geometry.
        return self.net(x)

In [4]:
# LOSS FUNCTIONS

def get_gradient(y, x):
    """Computes the gradient of y (N, 3) with respect to x (N, 3)."""
    grad_u = torch.autograd.grad(y[:, 0], x, grad_outputs=torch.ones_like(y[:, 0]), create_graph=True)[0]
    grad_v = torch.autograd.grad(y[:, 1], x, grad_outputs=torch.ones_like(y[:, 1]), create_graph=True)[0]
    grad_w = torch.autograd.grad(y[:, 2], x, grad_outputs=torch.ones_like(y[:, 2]), create_graph=True)[0]
    return torch.stack([grad_u, grad_v, grad_w], dim=1)


# Your loss_fn_strainenergy, slightly adapted
# NOTE: The function signature is simplified as it only needs the interior points.
def loss_fn_strainenergy(model, x_interior, E, nu):
    u_interior = model(x_interior)
    grad_u = get_gradient(u_interior, x_interior)

    epsilon_xx = grad_u[:, 0, 0]
    epsilon_yy = grad_u[:, 1, 1]
    epsilon_zz = grad_u[:, 2, 2]
    epsilon_xy = 0.5 * (grad_u[:, 0, 1] + grad_u[:, 1, 0])
    epsilon_xz = 0.5 * (grad_u[:, 0, 2] + grad_u[:, 2, 0])
    epsilon_yz = 0.5 * (grad_u[:, 1, 2] + grad_u[:, 2, 1])

    lmbda = (E * nu) / ((1 + nu) * (1 - 2 * nu))
    mu = E / (2 * (1 + nu))
    tr_epsilon = epsilon_xx + epsilon_yy + epsilon_zz

    sigma_xx = lmbda * tr_epsilon + 2 * mu * epsilon_xx
    sigma_yy = lmbda * tr_epsilon + 2 * mu * epsilon_yy
    sigma_zz = lmbda * tr_epsilon + 2 * mu * epsilon_zz
    sigma_xy = 2 * mu * epsilon_xy
    sigma_xz = 2 * mu * epsilon_xz
    sigma_yz = 2 * mu * epsilon_yz

    strain_energy_density = 0.5 * (
        sigma_xx * epsilon_xx + sigma_yy * epsilon_yy + sigma_zz * epsilon_zz +
        2 * (sigma_xy * epsilon_xy + sigma_xz * epsilon_xz + sigma_yz * epsilon_yz)
    )
    
    # NOTE: The domain_volume multiplication is removed. For complex shapes,
    # simply taking the mean of the density is standard practice for the loss.
    strain_energy = torch.mean(strain_energy_density)
    return strain_energy


# Your exact loss_fn_work
def loss_fn_work(model, x_forces, P_forces):
    u_at_forces = model(x_forces)
    work_potential = torch.sum(P_forces * u_at_forces)
    return work_potential


# Your loss_fn_bc, simplified for the new problem
# NOTE: The function signature is simplified as it only needs the boundary points.
def loss_fn_bc(model, x_boundary):
    u_boundary = model(x_boundary)
    loss_displacement = torch.mean(u_boundary.pow(2))

    # NOTE: The slope loss term is removed. For this problem, we are only
    # enforcing that the displacement is zero, not that the slope is zero.
    # grad_u_boundary = get_gradient(u_boundary, x_boundary)
    # loss_slope = torch.mean(grad_u_boundary.pow(2))
    # loss_bc = loss_displacement + loss_slope
    
    return loss_displacement

In [5]:
### FOR A VON MISES STRESS HEATMAP

def stress_visualisation(model, original_mesh):
    print("\n--- Starting Stress Visualization (Memory-Efficient) ---")
    
    # --- 1. Get all evaluation points from the original mesh ---
    #eval_points_np = original_mesh.points

    try:
        vis_mesh = pv.read('visualization_mesh.vtk')
        print(f"Loaded simplified mesh with {vis_mesh.n_points} points.")
    except FileNotFoundError:
        print("Error: 'visualization_mesh.vtk' not found. Please run the point generation script first.")
        return
    
    #print(f"Original mesh has {original_mesh.n_points} points.")
    #print("Simplifying mesh for visualization...")
    #vis_mesh = original_mesh.decimate(target_reduction=0.95)
    #print(f"Simplified mesh has {vis_mesh.n_points} points.")

    # --- 1. Get all evaluation points from the new, simplified mesh ---
    eval_points_np = vis_mesh.points
    
    # --- 2. Process points in batches to calculate stress ---
    print("Calculating stress field in batches...")
    model.eval() # Set the model to evaluation mode
    
    batch_size = 500  # You can adjust this size based on your available RAM
    von_mises_stress_list = []

    for i in range(0, len(eval_points_np), batch_size):
        # Get a small chunk of points
        points_batch_np = eval_points_np[i : i + batch_size]
        points_batch = torch.tensor(points_batch_np, dtype=torch.float32, requires_grad=True).to(next(model.parameters()).device)
        
        # --- Perform calculations only on the small batch ---
        u_pred = model(points_batch)
        grad_u = get_gradient(u_pred, points_batch)
        
        E = E_MATERIAL
        nu = NU_MATERIAL
        lmbda = (E * nu) / ((1 + nu) * (1 - 2 * nu))
        mu = E / (2 * (1 + nu))

        epsilon_xx = grad_u[:, 0, 0]
        epsilon_yy = grad_u[:, 1, 1]
        epsilon_zz = grad_u[:, 2, 2]
        epsilon_xy = 0.5 * (grad_u[:, 0, 1] + grad_u[:, 1, 0])
        epsilon_xz = 0.5 * (grad_u[:, 0, 2] + grad_u[:, 2, 0])
        epsilon_yz = 0.5 * (grad_u[:, 1, 2] + grad_u[:, 2, 1])
        tr_epsilon = epsilon_xx + epsilon_yy + epsilon_zz

        sigma_xx = lmbda * tr_epsilon + 2 * mu * epsilon_xx
        sigma_yy = lmbda * tr_epsilon + 2 * mu * epsilon_yy
        sigma_zz = lmbda * tr_epsilon + 2 * mu * epsilon_zz
        sigma_xy = 2 * mu * epsilon_xy
        sigma_xz = 2 * mu * epsilon_xz
        sigma_yz = 2 * mu * epsilon_yz

        term1 = (sigma_xx - sigma_yy)**2 + (sigma_yy - sigma_zz)**2 + (sigma_zz - sigma_xx)**2
        term2 = 6 * (sigma_xy**2 + sigma_yz**2 + sigma_xz**2)
        von_mises_stress_batch = torch.sqrt(0.5 * (term1 + term2)).cpu().detach().numpy()
        
        # Add the results for this batch to our list
        von_mises_stress_list.append(von_mises_stress_batch)
        
        print(f"\r  Processed {i + len(points_batch_np)} / {len(eval_points_np)} points...", end="")

    # Combine the results from all batches into a single array
    von_mises_stress = np.concatenate(von_mises_stress_list)
    print("\nStress calculation complete.")

    # --- 3. Add Stress Data to the Mesh and Plot ---
    mesh_with_stress = original_mesh.copy()
########################debugging print statements
    #print("\n--- Verifying Array Sizes Before Saving ---")
    #print(f"Number of points in mesh: {mesh_with_stress.n_points}")
    #print(f"Length of 'Von Mises Stress' array: {len(von_mises_stress)}")
########################
    mesh_with_stress['Von Mises Stress'] = von_mises_stress

    #print("Displaying stress plot...")
    #plotter_stress = pv.Plotter()
    #plotter_stress.add_mesh(mesh_with_stress, scalars='Von Mises Stress', cmap='plasma')
    #plotter_stress.camera_position = 'iso'
    #plotter_stress.add_text('Von Mises Stress', font_size=15)
    #plotter_stress.show()

    # --- 4. Save the Resulting Mesh to a VTK File ---
    output_filename = 'helmholtz_stress_result.vtk'
    mesh_with_stress.save(output_filename)
    print(f"\nStress results saved to {output_filename}. You can open this file in Paraview or other VTK viewers.")

In [6]:
# =============================================================================
# 3. TRAINING LOOP
# =============================================================================

# Initialize Model and Optimizer
model = PINN_Helmholtz()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4) # NOTE: A smaller learning rate is often more stable

print("\n--- Starting Training ---")
for epoch in range(4001): # NOTE: Reduced epochs for initial testing
    optimizer.zero_grad()
    
    loss_U = loss_fn_strainenergy(model, collocation_points, E_MATERIAL, NU_MATERIAL)
    loss_W = loss_fn_work(model, x_forces, P_forces)
    loss_BC = loss_fn_bc(model, fixed_boundary_points)
    

    total_loss = loss_U - 10 * loss_W + 1e6 * loss_BC
    
    total_loss.backward()
    optimizer.step()
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss = {total_loss.item():.6e}")
    
stress_visualisation(model, vis_mesh)



--- Starting Training ---
Epoch 0, Loss = 3.705531e+05
Epoch 500, Loss = -3.256535e+03
Epoch 1000, Loss = -4.626090e+03
Epoch 1500, Loss = -5.656691e+03
Epoch 2000, Loss = -1.174883e+04
Epoch 2500, Loss = -3.401733e+05
Epoch 3000, Loss = -9.035688e+05
Epoch 3500, Loss = -1.314292e+06
Epoch 4000, Loss = -1.655478e+06

--- Starting Stress Visualization (Memory-Efficient) ---
Loaded simplified mesh with 4532 points.
Calculating stress field in batches...
  Processed 4532 / 4532 points...
Stress calculation complete.

Stress results saved to helmholtz_stress_result.vtk. You can open this file in Paraview or other VTK viewers.
